In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from random import choice
pd.options.mode.chained_assignment = None 
import pickle

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser('~'), 'Documents/Coding/Python'))
# sys.path

from lonelyboy.geospatial import plots as gsplt
from lonelyboy.geospatial import preprocessing as gspp
from lonelyboy.timeseries import lbtimeseries as tspp
# from lonelyboy.geospatial import group_patterns as gsgp


# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import PyQt5
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

get_ipython().magic('matplotlib qt')
# get_ipython().magic('matplotlib inline')

In [89]:
# PLT_FIG_WIDTH = 10.487
PLT_FIG_WIDTH = 3.787
PLT_FIG_HEIGHT = PLT_FIG_WIDTH / 1.618

In [4]:
ports = pd.read_pickle('./data/pkl/ports_raw.pkl')
traj  = pd.read_csv('./data/csv/nari_dynamic_test.csv')
# traj  = pd.read_csv('./data/csv/nari_dynamic.csv')

traj  = gspp.gdf_from_df(traj, crs={'init':'epsg:4326'})
ports = gspp.create_port_bounds(ports, epsg=2154, port_radius=2000)

In [6]:
gsplt.map_plot(traj, color=['steelblue'], title='Testing Trajectories', fontsize=10)

## Section 2: Distribution over Port Popularity

In [7]:
port_popularity = pd.DataFrame([], columns=['port_id', '#arrivals_departures'])

In [8]:
%%time
sindex = traj.sindex

port_popularity = pd.DataFrame([], columns=['port_id', '#arrivals_departures'])
# find the points that intersect with each subpolygon and add them to points_within_geometry
for (port_id, poly) in zip(ports.gid, ports.geom):
    # find approximate matches with r-tree, then precise matches from those approximate ones
    possible_matches_index = list(sindex.intersection(poly.bounds))
    possible_matches = traj.iloc[possible_matches_index]
    precise_matches  = possible_matches[possible_matches.intersects(poly)]
    port_popularity  = port_popularity.append(pd.DataFrame([[port_id, len(precise_matches)]], columns=['port_id', '#arrivals_departures']))
    
port_popularity.reset_index(inplace=True, drop=True)

Wall time: 5.68 s


* ### Reading CSVs

In [12]:
data_folder = './data/csv/stats/port_popularity_report'
for cnt, file in enumerate(os.listdir(data_folder)):
    tmp = pd.read_csv(os.path.join(data_folder, file))
    if cnt==0:
        port_popularity = tmp
    else: 
        port_popularity['#arrivals_departures'] = port_popularity['#arrivals_departures'] + tmp['#arrivals_departures']

In [ ]:
port_popularity

In [45]:
port_popularity.to_csv('./data/csv/stats/port_popularity_report.csv', index=False, header=True)

In [5]:
port_popularity = pd.read_csv('./data/csv/stats/port_popularity_report.csv')

In [6]:
port_popularity_merged = pd.merge(ports, port_popularity, how='inner', left_on='gid', right_on='port_id')
port_popularity_merged.head()

,gid,gml_id,por_id,libelle_po,insee_comm,por_x,por_y,geom,port_id,#arrivals_departures
0,1,port.1,1,Le Vivier-sur-Mer,35361,297025.0,2408370.0,POLYGON ((-1.744722258976547 48.60382613452489...,1,0
1,2,port.10,10,Saint-Samson sur Rance,22327,279335.0,2396060.0,POLYGON ((-1.974979769268805 48.48483583379186...,2,0
2,3,port.100,100,Douarnenez,29046,103135.0,2365330.0,POLYGON ((-4.314453220845626 48.09876366555108...,3,887769
3,4,port.101,101,Brézellec,29028,79105.4,2364190.0,POLYGON ((-4.634389885959526 48.06978148591438...,4,7342
4,5,port.102,102,Sein,29083,64562.5,2362180.0,"POLYGON ((-4.82623989311881 48.04003666854432,...",5,182025


In [7]:
tmp = port_popularity_merged.sort_values(['#arrivals_departures'], ascending=False).iloc[:2,:]
port_popularity_merged.sort_values(['#arrivals_departures'], ascending=False)

,gid,gml_id,por_id,libelle_po,insee_comm,por_x,por_y,geom,port_id,#arrivals_departures
207,208,port.86,86,Brest,29019,95931.3,2397840.0,POLYGON ((-4.448417803028764 48.38443172631552...,208,7297716
2,3,port.100,100,Douarnenez,29046,103135.0,2365330.0,POLYGON ((-4.314453220845626 48.09876366555108...,3,887769
208,209,port.87,87,Brest Moulin Blanc,29019,99011.9,2398830.0,POLYGON ((-4.408135980899162 48.39566629117813...,209,284938
4,5,port.102,102,Sein,29083,64562.5,2362180.0,"POLYGON ((-4.82623989311881 48.04003666854432,...",5,182025
219,220,port.97,97,Camaret,29022,86070.5,2386660.0,POLYGON ((-4.567755996722108 48.27657284982146...,220,178939
220,221,port.98,98,Le Fret,29042,92437.7,2387210.0,"POLYGON ((-4.48296706537295 48.2865582380885, ...",221,152680
221,222,port.99,99,Morgat,29042,92375.0,2380350.0,POLYGON ((-4.475798890467401 48.22499497743777...,222,33503
206,207,port.85,85,Le Conquet,29040,73575.0,2397180.0,POLYGON ((-4.748144371917133 48.36084100244644...,207,27524
218,219,port.96,96,Roscanvel,29238,89850.0,2390640.0,POLYGON ((-4.521715842237255 48.31520244875784...,219,25142
205,206,port.84,84,Île de Molène,29084,60462.5,2402630.0,POLYGON ((-4.931072710007551 48.39882861615506...,206,23012


In [78]:
_ = map_plot(ports, tmp, figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT), column='cmap', color=['black', 'red'], legend=False)

In [60]:
tmp_popular_ports = port_popularity_merged.sort_values(['#arrivals_departures'], ascending=False).iloc[:2,:]
tmp_popular_ports = gspp.create_port_bounds(tmp_popular_ports)
tmp_popular_ports

,gid,gml_id,por_id,libelle_po,insee_comm,por_x,por_y,geom,port_id,#arrivals_departures,cmap
207,208,port.86,86,Brest,29019,95931.3,2397840.0,POLYGON ((-4.421680857347681 48.38700041737251...,208,7297716,6.863187
2,3,port.100,100,Douarnenez,29046,103135.0,2365330.0,POLYGON ((-4.287854236200064 48.10130266698893...,3,887769,5.948300


In [74]:
# import matplotlib
import contextily as ctx
from sklearn.preprocessing import MinMaxScaler


def map_plot(df1, df2 = None, ax=None, title=None, fontsize=25, color=['r', 'g'], figsize=(15,15), attribution="", **kwargs):
    '''
    Plot one or two dataframes on top of eachother.

    TODO - Add support for N Dataframes and more parameters, other that figsize.
    '''
    df1.crs = {'init': 'epsg:4326'}
    
    if ax is not None:
        df1.to_crs(epsg=3857).plot(figsize=figsize,color=color[0], ax=ax, **kwargs)
    else:
        ax = df1.to_crs(epsg=3857).plot(figsize=figsize,color=color[0], **kwargs)
    if title is not None:
        ax.set_title(title, fontsize=fontsize)
    if df2 is not None:
        df2.crs = {'init': 'epsg:4326'}
        df2.to_crs(epsg=3857).plot(figsize=figsize,color=color[1], ax=ax, **kwargs)
    ctx.add_basemap(ax, attribution=attribution)
    ax.margins(0)
    ax.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
    
    return ax

# scaler = MinMaxScaler(feature_range=(0,1))
# port_popularity_merged['cmap'] = scaler.fit_transform(port_popularity.values[:,1].reshape(-1,1))
# fig, ax = plt.subplots(1, figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT))

port_popularity_merged['cmap'] = port_popularity_merged['#arrivals_departures'].apply(np.log10).replace(-np.inf, 0)
_ = map_plot(port_popularity_merged, figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT), column='cmap', color=[None], cmap='viridis', legend=True)


# sm = plt.c;m.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=port_popularity_merged['cmap'].min(), vmax=port_popularity_merged['cmap'].max()))
# # empty array for the data range
# sm._A = []
# # add the colorbar to the figure
# cbar = fig.colorbar(sm, orientation='horizontal', shrink=0.6, pad=0.01)
# cbar.set_label(label='$\log_{10} (\#arrivals\_departures)$',size=5,weight='bold')

cax = plt.gcf().get_axes()[1]
#and we can modify it, i.e.:
cax.set_ylabel(r'\textbf{$\log_{10} (\#arrivals\_departures)$}', fontsize=8)
cax.tick_params(labelsize=8)
cax.xaxis.set_ticks_position("bottom")

plt.title(r'\textbf{Port Popularity Heatmap}', fontsize=8)
plt.axis('off')
plt.show()
# gsplt.map_plot(port_popularity_merged, column='#arrivals_departures', color=[None], cmap='viridis', title='Port Popularity', fontsize=10, legend=True)
# plt.savefig(os.path.join('.', 'experimental_figures', 'figs_final', 'Port_Popularity.pdf'), dpi=500, bbox_inches='tight', pad_inches=0) 
plt.savefig(os.path.join('..', 'Port_Popularity_V4.pdf'), dpi=500, bbox_inches='tight') 

In [79]:
# scaler = MinMaxScaler(feature_range=(0,1))
# port_popularity_merged['cmap'] = scaler.fit_transform(port_popularity.values[:,1].reshape(-1,1))
# fig, ax = plt.subplots(1, figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT))

port_popularity_merged['cmap'] = port_popularity_merged['#arrivals_departures'].apply(np.log10).replace(-np.inf, 0)
ax = map_plot(port_popularity_merged, figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT), column='cmap', color=[None], cmap='viridis', legend=True)
tmp_popular_ports.to_crs(epsg=3857).plot(facecolor='none', edgecolor='red', lw=0.7, ax=ax)

# sm = plt.c;m.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=port_popularity_merged['cmap'].min(), vmax=port_popularity_merged['cmap'].max()))
# # empty array for the data range
# sm._A = []
# # add the colorbar to the figure
# cbar = fig.colorbar(sm, orientation='horizontal', shrink=0.6, pad=0.01)
# cbar.set_label(label='$\log_{10} (\#arrivals\_departures)$',size=5,weight='bold')

cax = plt.gcf().get_axes()[1]
#and we can modify it, i.e.:
cax.set_ylabel(r'\#AIS Points Inside Port', fontsize=8)
cax.set_yticklabels([r'$0$', r'$10^1$', r'$10^2$', r'$10^3$', r'$10^4$', r'$10^5$', r'$10^6$'])  # horizontal colorbar
cax.tick_params(labelsize=8)
cax.xaxis.set_ticks_position("bottom")


# plt.title(r'\textbf{Port Popularity Heatmap}', fontsize=8)
plt.axis('off')
plt.show()
# gsplt.map_plot(port_popularity_merged, column='#arrivals_departures', color=[None], cmap='viridis', title='Port Popularity', fontsize=10, legend=True)
# plt.savefig(os.path.join('.', 'experimental_figures', 'figs_final', 'Port_Popularity.pdf'), dpi=500, bbox_inches='tight', pad_inches=0) 
plt.savefig(os.path.join('..', 'Port_Popularity_V5.pdf'), dpi=500, bbox_inches='tight', pad_inches=0)

In [91]:
port_popularity_merged['cmap'] = port_popularity_merged['#arrivals_departures'].apply(np.log10).replace(-np.inf, 0)

PLT_FIG_WIDTH2 = 2.787
PLT_FIG_HEIGHT2 = PLT_FIG_WIDTH2 / 1.618

fig, ax = plt.subplots(1, figsize=(PLT_FIG_WIDTH2, PLT_FIG_HEIGHT2))
_ = gsplt.map_plot(port_popularity_merged, ax=ax, column='cmap', color=[None], cmap='viridis', legend=False)

sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=port_popularity_merged['cmap'].min(),
                                                               vmax=port_popularity_merged['cmap'].max()))
# empty array for the data range
sm._A = []
# add the colorbar to the figure
cbar = fig.colorbar(sm, orientation='horizontal', shrink=0.6, pad=0.05)
# cbar.set_label(label='$\log_{10} (\#arrivals\_departures)$',size=8,weight='bold')


cax = plt.gcf().get_axes()[1]
#and we can modify it, i.e.:
cax.tick_params(labelsize=5)
cax.set_xlabel(r'\#AIS Points Inside Port', fontsize=5)
cax.set_xticklabels([r'$0$', r'$10^2$', r'$10^4$', r'$10^6$'])  # horizontal colorbar
cax.tick_params(labelsize=5)
cax.xaxis.set_ticks_position("bottom")

plt.suptitle(r'', fontsize=5)
plt.axis('off')
plt.show()
# gsplt.map_plot(port_popularity_merged, column='#arrivals_departures', color=[None], cmap='viridis', title='Port Popularity', fontsize=10, legend=True)
# plt.savefig(os.path.join('.', 'experimental_figures', 'figs_final', 'Port_Popularity.pdf'), dpi=500, bbox_inches='tight', pad_inches=0) 
plt.savefig(os.path.join('..', 'Port_Popularity_V5_horizontal.pdf'), dpi=500, bbox_inches='tight', pad_inches=0) 

## Section 3: Distribution of AIS Activity
  * ### Outside Port

In [4]:
def vessel_ais_activity(vessel, ports, port_epsg=2154, port_radius=2000, temporal_threshold=12):                                               
    port_bounds = gspp.create_port_bounds(ports, epsg=port_epsg, port_radius=port_radius)
    port_segmented_trajectories = gspp.segment_trajectories_v2(vessel, port_bounds)
    temporal_segmented_trajectories = gspp.__temporal_segment(port_segmented_trajectories, temporal_threshold=temporal_threshold)
    vessel_fn = pd.concat(temporal_segmented_trajectories, ignore_index=True)
    vessel_fn.sort_values('ts', inplace=True)
    vessel_fn.drop(['index'], axis=1, inplace=True)
    return vessel_fn

In [5]:
traj_seg = traj.groupby('mmsi', group_keys=False).apply(vessel_ais_activity, ports)
traj_seg.sort_values('ts', inplace=True)

Vessel mmsi:205067000
Segments Before: 1
Segments After: 2
Vessel mmsi:205067000
Segments Before: 1
Segments After: 2
Vessel mmsi:205067000
Segments Before: 1
Segments After: 2
Vessel mmsi:227519920
Segments Before: 55
Segments After: 55
Vessel mmsi:228037600
Segments Before: 136
Segments After: 139
Vessel mmsi:305166000
Segments Before: 4
Segments After: 5


In [22]:
gsplt.map_plot(traj_seg, color=[None], column='traj_id', cmap='tab20',  title='Testing Trajectories Segments', fontsize=10, legend=True)

In [41]:
ais_activity_outside_port = traj_seg.groupby(['mmsi', 'traj_id_12h_gap', pd.to_datetime(traj_seg.ts, unit='s').dt.date], group_keys=False).apply(lambda df: df.ts.diff().sum()/3600).to_frame().reset_index()
ais_activity_outside_port.columns = ['mmsi', 'traj_id_temporal_gap','date', '#hrs']
ais_activity_outside_port = ais_activity_outside_port.groupby(['mmsi', 'date'], group_keys=False).apply(lambda df: df['#hrs'].sum()).to_frame().reset_index()
ais_activity_outside_port.columns = ['mmsi', 'date', '#hrs']
ais_activity_outside_port

,mmsi,date,#hrs
0,205067000,2015-10-22,1.843333
1,205067000,2015-10-23,21.251389
2,205067000,2016-02-10,1.249444
3,227519920,2015-10-01,4.116667
4,227519920,2015-10-03,2.107778
5,227519920,2015-10-09,0.116667
6,227519920,2015-10-10,0.484167
7,227519920,2015-10-11,0.174722
8,227519920,2015-10-18,3.883333
9,227519920,2015-10-19,2.115833


  * ### Within Port

In [123]:
ais_activity_within_port = pd.DataFrame([], columns=['mmsi', 'date', '#hrs'])

for groupby_cols, mmsi_traj in traj_seg.groupby(['mmsi', pd.to_datetime(traj_seg.ts, unit='s').dt.date]):
    breaking_points = mmsi_traj.loc[mmsi_traj.traj_id.diff() == 1].index.tolist()
#     print ('Vessel: ', groupby_cols[0])
#     print ('Breaking Points:\n', breaking_points)
    if (len(breaking_points) == 0):
        ais_activity_within_port = ais_activity_within_port.append(pd.DataFrame([[groupby_cols[0], groupby_cols[1], 0]], columns=['mmsi', 'date', '#hrs']))
        continue
        
    hrs = np.around(np.sum([mmsi_traj.loc[bp-1:bp,:].ts.diff().values[1] for bp in breaking_points])/3600, 3)
    ais_activity_within_port = ais_activity_within_port.append(pd.DataFrame([[groupby_cols[0], groupby_cols[1], hrs]], columns=['mmsi', 'date', '#hrs']))

ais_activity_within_port.reset_index(drop=True, inplace=True)

In [124]:
ais_activity_within_port

,mmsi,date,#hrs
0,205067000,2015-10-22,0
1,205067000,2015-10-23,0
2,205067000,2016-02-10,0
3,227519920,2015-10-01,0.232
4,227519920,2015-10-03,0.101
5,227519920,2015-10-09,0
6,227519920,2015-10-10,0
7,227519920,2015-10-11,0
8,227519920,2015-10-18,0.05
9,227519920,2015-10-19,0.226


# To Use on the Script

In [70]:
for mmsi, ves_traj in traj.groupby('mmsi', group_keys=False):
    ves_traj = vessel_activity_outside_port(ves_traj, ports)
    ais_activity_outside_port = ves_traj.groupby(['mmsi', 'traj_id_12h_gap', pd.to_datetime(ves_traj.ts, unit='s').dt.date], group_keys=False).apply(lambda df: df.ts.diff().sum()/3600).to_frame().reset_index()
    ais_activity_outside_port.columns = ['mmsi', 'traj_id_temporal_gap','date', '#hrs']
    print(ais_activity_outside_port.head())

Vessel mmsi:205067000
Segments Before: 1
Segments After: 2
        mmsi  traj_id_temporal_gap        date       #hrs
0  205067000                     0  2015-10-22   1.843333
1  205067000                     0  2015-10-23  21.251389
2  205067000                     1  2016-02-10   1.249444
Vessel mmsi:227519920
Segments Before: 55
Segments After: 55
        mmsi  traj_id_temporal_gap        date      #hrs
0  227519920                     0  2015-10-01  0.573889
1  227519920                     1  2015-10-01  0.400000
2  227519920                     2  2015-10-01  2.275000
3  227519920                     3  2015-10-01  0.350000
4  227519920                     4  2015-10-01  0.258889
Vessel mmsi:228037600
Segments Before: 136
Segments After: 139
        mmsi  traj_id_temporal_gap        date      #hrs
0  228037600                     0  2015-09-30  1.197222
1  228037600                     1  2015-09-30  0.791389
2  228037600                     1  2015-10-01  1.225000
3  228037600   

In [58]:
# tmp = points_within_geometry.groupby(['mmsi', pd.to_datetime(points_within_geometry.ts, unit='s').dt.date], group_keys=False).apply(lambda df: df.ts.diff().sum()/3600).to_frame().reset_index()
# tmp.columns = ['mmsi', 'date', '#hrs']
# tmp

# gsplt.map_plot(points_within_geometry, points_outside_geometry, color=['r','steelblue'], title='Points Outside/Inside Port Radius', fontsize=10)

## Read CSVs And Make Plots

In [9]:
data_folder = './data/csv/stats/nari_dynamic_ais_activity_report/'

nari_dynamic_ais_activity_report_outside_port = []
nari_dynamic_ais_activity_report_within_port = []

for cnt, file in enumerate(os.listdir(data_folder)):
    tmp = pd.read_csv(os.path.join(data_folder, file))
    if 'outside' in file: 
        print (f'\'outside\' in: {file}')
        nari_dynamic_ais_activity_report_outside_port.append(tmp)
    elif 'within' in file: 
        print (f'\'within\' in: {file}')
        nari_dynamic_ais_activity_report_within_port.append(tmp)
        
nari_dynamic_ais_activity_report_outside_port = pd.concat(nari_dynamic_ais_activity_report_outside_port, ignore_index=True)
nari_dynamic_ais_activity_report_within_port = pd.concat(nari_dynamic_ais_activity_report_within_port, ignore_index=True)

nari_dynamic_ais_activity_report_outside_port.sort_values(['date'], inplace=True)
nari_dynamic_ais_activity_report_within_port.sort_values(['date'], inplace=True)

nari_dynamic_ais_activity_report_outside_port.reset_index(drop=True, inplace=True)
nari_dynamic_ais_activity_report_within_port.reset_index(drop=True, inplace=True)

'outside' in: nari_dynamic_ais_activity_report_outside_port.csv
'outside' in: nari_dynamic_ais_activity_report_outside_port_0.csv
'outside' in: nari_dynamic_ais_activity_report_outside_port_1.csv
'outside' in: nari_dynamic_ais_activity_report_outside_port_2.csv
'outside' in: nari_dynamic_ais_activity_report_outside_port_3.csv
'outside' in: nari_dynamic_ais_activity_report_outside_port_4.csv
'outside' in: nari_dynamic_ais_activity_report_outside_port_5.csv
'within' in: nari_dynamic_ais_activity_report_within_port.csv
'within' in: nari_dynamic_ais_activity_report_within_port_0.csv
'within' in: nari_dynamic_ais_activity_report_within_port_1.csv
'within' in: nari_dynamic_ais_activity_report_within_port_2.csv
'within' in: nari_dynamic_ais_activity_report_within_port_3.csv
'within' in: nari_dynamic_ais_activity_report_within_port_4.csv
'within' in: nari_dynamic_ais_activity_report_within_port_5.csv


In [53]:
nari_dynamic_ais_activity_report_outside_port.to_csv('./data/csv/stats/nari_dynamic_ais_activity_report_outside_port.csv', index=False, header=True)
nari_dynamic_ais_activity_report_within_port.to_csv('./data/csv/stats/nari_dynamic_ais_activity_report_within_port.csv', index=False, header=True)

In [60]:
tmp = nari_dynamic_ais_activity_report_outside_port.merge(nari_dynamic_ais_activity_report_within_port, how='outer', left_on=['mmsi','date'], right_on = ['mmsi','date'], suffixes=('_outside_port', '_within_port'))
tmp.to_csv('./data/csv/stats/nari_dynamic_ais_activity_report.csv', index=False, header=True)

In [11]:
tmp = pd.read_csv('./data/csv/stats/nari_dynamic_ais_activity_report.csv')

In [12]:
tmp.head()

,mmsi,date,#hrs_outside_port,#hrs_within_port
0,566838000,2015-09-30,0.000,0.0
1,304091000,2015-09-30,1.950,0.0
2,228218700,2015-09-30,1.351,0.0
3,228160000,2015-09-30,0.433,0.0
4,228131600,2015-09-30,1.993,0.0


In [13]:
ais_activity_by_date = tmp.groupby([pd.to_datetime(tmp.date).dt.to_period('D')]).apply(lambda df: pd.DataFrame([[df['#hrs_outside_port'].sum(), df['#hrs_within_port'].sum()]])).reset_index().drop(['level_1'], axis=1)
ais_activity_by_date.columns = ['date', '#Hours Outside Port', '#Hours Within Port']
ais_activity_by_date.head()

,date,#Hours Outside Port,#Hours Within Port
0,2015-09-30,52.137,0.000
1,2015-10-01,1042.867,157.307
2,2015-10-02,896.200,7.719
3,2015-10-03,1136.994,6.348
4,2015-10-04,622.045,0.895


In [32]:
ais_activity_by_date.plot.bar(cmap='tab20', x='date', figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT), width=0.95, rot=0)
plt.show()

In [14]:
avg_ais_activity_by_date = tmp.groupby([tmp.date]).apply(lambda df: pd.DataFrame([[df['#hrs_outside_port'].mean(), df['#hrs_within_port'].mean()]])).reset_index().drop(['level_1'], axis=1)
avg_ais_activity_by_date.columns = ['date', 'Avg. #Hours Outside Port', 'Avg. #Hours Within Port']
avg_ais_activity_by_date.head()

,date,Avg. #Hours Outside Port,Avg. #Hours Within Port
0,2015-09-30,1.212488,0.000000
1,2015-10-01,5.793706,0.873928
2,2015-10-02,5.464634,0.047067
3,2015-10-03,5.952848,0.033236
4,2015-10-04,7.149943,0.010287


In [15]:
ais_activity_by_mmsi = tmp.groupby(['mmsi', pd.to_datetime(tmp.date).dt.to_period('D')]).apply(lambda df: pd.DataFrame([[df['#hrs_outside_port'].sum(), df['#hrs_within_port'].sum(), 24-df['#hrs_outside_port'].sum()+df['#hrs_within_port'].sum()]])).reset_index().drop(['level_2'], axis=1)
ais_activity_by_mmsi = ais_activity_by_mmsi.groupby(['mmsi']).apply(lambda df: pd.DataFrame([[df[0].sum(), df[1].sum(), df[2].sum()]])).reset_index().drop(['level_1'], axis=1)
ais_activity_by_mmsi.columns = ['mmsi', '#Hours Outside Port', '#Hours Within Port', '#Hours AIS Device OFF']
ais_activity_by_mmsi

,mmsi,#Hours Outside Port,#Hours Within Port,#Hours AIS Device OFF
0,37100300,35.472,0.031,36.559
1,205067000,25.202,0.000,46.798
2,205204000,84.369,0.081,155.712
3,205209000,1.314,0.000,22.686
4,205277430,4.618,0.000,43.382
5,205309000,22.111,0.019,49.908
6,205439000,62.456,0.000,129.544
7,205489000,8.724,0.000,39.276
8,205656000,8.035,0.000,63.965
9,205688000,9.146,0.000,14.854


In [16]:
labels = ['Outside Port', 'Within Port', 'AIS Device OFF']
df = pd.DataFrame({'#Hours': ais_activity_by_mmsi.iloc[:, 1:].sum().values}, 
             index=labels)

explode = (0.2, 0, 0)  
ax = df.plot.pie(y='#Hours', 
                   figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT), 
                   labels=[r'\textbf{Outside Port}', r'\textbf{Within Port}', r'\textbf{AIS Device OFF}'],
                   explode=explode,
                   startangle=90,
#                    autopct='%.1f%%',
                   autopct = lambda p: '{:.2f}\%'.format(p),
                   pctdistance=0.70,
                   cmap='tab20c',
                   legend=False)
ax.axis('equal')

for text in ax.texts:
    text.set_fontsize(8)
    if ' ' in text.get_text():
        text.set_color('grey')
    print(text)

plt.ylabel('')
plt.savefig(os.path.join('.', 'experimental_figures', 'figs_final', 'AIS_Activity_Pie_Chart.pdf'), dpi=300, bbox_inches='tight')

Text(-0.9165934630017915, 0.921876577195876, '\\textbf{Activity outside port}')
Text(-0.6345647051550863, 0.638222245750991, '24.91\\%')
Text(-1.099681758845238, -0.026458066124407972, '\\textbf{Activity inside port}')
Text(-0.6997974829015151, -0.0168369511700778, '0.95\\%')
Text(0.7984785785060152, -0.7565923338674623, '\\textbf{Unknown (or in-) activity}')
Text(0.5081227317765551, -0.48146784882474863, '74.14\\%')


In [9]:
avg_ais_activity_by_vessel = tmp.groupby(['mmsi']).apply(lambda df: pd.DataFrame([[df['#hrs_outside_port'].mean(), df['#hrs_within_port'].mean()]])).reset_index().drop(['level_1'], axis=1)
avg_ais_activity_by_vessel.columns = ['mmsi', '#Hours Outside Port', '#Hours Within Port'] 
ax = avg_ais_activity_by_vessel.plot.bar(cmap='tab20', figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT), width=0.95, rot=0)

In [19]:
avg_ais_activity_by_week = tmp.groupby([pd.to_datetime(tmp.date).dt.to_period('W')]).apply(lambda df : pd.DataFrame([[df['#hrs_outside_port'].mean(), df['#hrs_within_port'].mean()]])).reset_index().drop(['level_1'], axis=1)
avg_ais_activity_by_week.columns = ['date', '\#Hours Outside Port', '\#Hours Within Port'] 
print (avg_ais_activity_by_week.head())

ax = avg_ais_activity_by_week.plot.bar(cmap='tab20', figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT), fontsize=8, width=1, rot=65, legend=False)
# Put a legend below current axis
plt.title(r'\textbf{Avg. AIS Activity per Week}', fontsize=8, y=1)
plt.xlabel(r'\textbf{Week Number (Start:01-10-2015, End:31-03-2016)}', fontsize=8)
plt.ylabel(r'\textbf{\#Hours}', fontsize=8)

ax.legend(fontsize='small', loc='lower center', bbox_to_anchor=(0.5, -0.4),
          fancybox=False, shadow=False, frameon=False, ncol=2)

plt.savefig(os.path.join('.', 'experimental_figures', 'figs_final', 'Avg_AIS_Activity_by_Week.pdf'), dpi=300, bbox_inches='tight')

                    date  \#Hours Outside Port  \#Hours Within Port
0  2015-09-28/2015-10-04              5.639463             0.259051
1  2015-10-05/2015-10-11              7.047643             0.366189
2  2015-10-12/2015-10-18              6.141510             0.717253
3  2015-10-19/2015-10-25              6.176943             0.344040
4  2015-10-26/2015-11-01              5.020846             0.089937


In [20]:
cumulative_ais_activity_by_week = tmp.groupby([pd.to_datetime(tmp.date).dt.to_period('W')]).apply(lambda df : pd.DataFrame([[df['#hrs_outside_port'].sum(), df['#hrs_within_port'].sum()]])).reset_index().drop(['level_1'], axis=1)
cumulative_ais_activity_by_week.columns = ['week', '\#Hours Outside Port', '\#Hours Within Port'] 

ax = cumulative_ais_activity_by_week.plot.bar(cmap='tab20', figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT), fontsize=8, width=1, rot=65)

plt.title(r'\textbf{Cumulative AIS Activity per Week}', fontsize=8, y=1)
plt.xlabel(r'\textbf{Week Number (Start:01-10-2015, End:31-03-2016)}', fontsize=8)
plt.ylabel(r'\textbf{\#Hours}', fontsize=8)

ax.legend(fontsize='small', loc='lower center', bbox_to_anchor=(0.5, -0.4),
          fancybox=False, shadow=False, frameon=False, ncol=2)

plt.savefig(os.path.join('.', 'experimental_figures', 'figs_final', 'Cumulative_AIS_Activity_by_Week.pdf'), dpi=300, bbox_inches='tight')